In [3]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

2023-02-15 18:16:43.906085: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#Initialize
train_dir = 'data/train'
val_dir = 'data/test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


Now we can build the convolution network architecture to train the data:

In [5]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

2023-02-14 18:25:09.528232: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
#Compile and train the model:

#emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
#emotion_model_info = emotion_model.fit_generator(
#        train_generator,
#        steps_per_epoch=28709 // 64,
#        epochs=30,
#        validation_data=validation_generator,
#        validation_steps=7178 // 64)

/var/folders/hp/m_3sfw2d3gnd52gvxwg38bk00000gn/T/ipykernel_33218/1130765543.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/30
448/448 [==============================] - 210s 466ms/step - loss: 1.7980 - accuracy: 0.2647 - val_loss: 1.6980 - val_accuracy: 0.3449
Epoch 2/30
448/448 [==============================] - 239s 533ms/step - loss: 1.6272 - accuracy: 0.3646 - val_loss: 1.5473 - val_accuracy: 0.4129
Epoch 3/30
448/448 [==============================] - 233s 520ms/step - loss: 1.5320 - accuracy: 0.4103 - val_loss: 1.4763 - val_accuracy: 0.4379
Epoch 4/30
448/448 [==============================] - 194s 432ms/step - loss: 1.4593 - accuracy: 0.4421 - val_loss: 1.3936 - val_accuracy: 0.4706
Epoch 5/30
448/448 [==============================] - 179s 398ms/step - loss: 1.3945 - accuracy: 0.4655 - val_loss: 1.3415 - val_accuracy: 0.4869
Epoch 6/30
448/448 [==============================] - 190s 425ms/step - loss: 1.3377 - accuracy: 0.4942 - val_loss: 1.2944 - val_accuracy: 0.5078
Epoch 7/30
448/448 [==============================] - 183s 408ms/step - loss: 1.2951 - accuracy: 0.5082 - val_loss: 1.2694 -

Now save the model:

In [8]:
emotion_model.save_weights('model.h5')

In [6]:
emotion_model.load_weights('model.h5')

In [7]:
cv2.ocl.setUseOpenCL(False)

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 52ms/step


1/1 [==============================] - 0s 24ms/step


KeyboardInterrupt: 